In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time

In [2]:
def crawler_clubs(team, year):
    pref = "http://www.worldfootball.net/teams/"
    url = pref + team + "/" + str(year) + "/3/"
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
    }
    data = requests.get(url, headers = header).content
    soup = BeautifulSoup(data, 'html.parser')
    data_raw = soup.find("table", attrs={"class": "standard_tabelle"})
    items = data_raw.find_all("tr")
    df = pd.DataFrame(index=np.arange(len(items)), columns=[
            'match_type','date','home', 'away', 'score', 'points'])
    count = 0
    for item in items:
        is_mtype = item.find("a",attrs = {"href": re.compile('^\/all_matches')})
        if is_mtype:
            mtype = is_mtype.getText()
            continue
        tmp = item.find_all('td')
        if len(tmp) > 6:
            a, b, c, d = tmp[1], tmp[3], tmp[5], tmp[6]
            if a and b and c and d:
                if d.getText() != "\n-:- \n":
                    df.loc[count] = [mtype, a.getText(), b.getText(), 
                                     c.getText().strip('\n\r\t '), d.getText().strip('\n\r\t '), np.NaN]
                    count += 1
    
    df = df[df.home.notnull()]
    for i in xrange(len(df)):
        tmp = df.score[i]
        if tmp[0] > tmp[2]:
            df.points[i] = 3
        elif tmp[0] == tmp[2]:
            df.points[i] = 1
        else:
            df.points[i] = 0

    return df

In [5]:
teams = ["arsenal-fc", "chelsea-fc", "liverpool-fc",
        "manchester-city", "manchester-united", "tottenham-hotspur"]
years = np.arange(1993, 2018)

In [46]:
ti = teams[2]

In [39]:
ti = "real-madrid"

In [47]:
for year in years:
    print year,
    df_tmp = crawler_clubs(ti, year)
    t = np.random.rand(1)*2
    time.sleep(t)
    address = 'data/'+ti+'/'+ti+'_'+str(year-1)+'_'+str(year)+'.csv'
    df_tmp.to_csv(address, encoding='utf-8')

1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017


In [43]:
klopp_15_16 = pd.read_csv('data/liverpool-fc/liverpool-fc_2015_2016.csv', index_col=0)
klopp_16_17 = pd.read_csv('data/liverpool-fc/liverpool-fc_2016_2017.csv', index_col=0)

In [45]:
klopp_15_16.head()

,match_type,date,home,away,score,points
0,Europa League 2015/2016,17/09/2015,A,Girondins Bordeaux,1:1 (0:0),1
1,Europa League 2015/2016,01/10/2015,H,FC Sion,1:1 (1:1),1
2,Europa League 2015/2016,22/10/2015,H,Rubin Kazan,1:1 (1:1),1
3,Europa League 2015/2016,05/11/2015,A,Rubin Kazan,1:0 (0:0),3
4,Europa League 2015/2016,26/11/2015,H,Girondins Bordeaux,2:1 (2:1),3
